In [3]:
import pandas as pd
import urllib.parse
from random import shuffle
from my_rymscraper.rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [4]:
mt = pd.read_csv("meta.csv")

In [5]:
# mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"];
lower_alnum = lambda s: "".join(x for x in s.lower() if x.isalnum())
mt["id"] = mt["Singer"].apply(lower_alnum) + mt["Release_Year"].apply(str) + mt["Album_Name"].apply(lambda s: lower_alnum(s)[:3] + lower_alnum(s)[-3:])
tags = list(zip(mt["id"], mt["Singer"], mt["Album_Name"]))
# tags = ["Common - A Beautiful Revolution, Vol. 2"]
cols = ['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages', 'Ranked']
df = pd.DataFrame(columns=cols)

In [24]:
lower_alnum_dash = lambda s: "".join(x for x in s.replace(" ", "-").lower() if (x.isalnum() or x == "-" ))
tags = [('stevereich2018pultet', 'Steve Reich', 'Pulse/Quartet'),
 ('christinavantzou2018no4no4', 'Christina Vantzou', 'No. 4'),
 ('billcallahan2022ytiaer', 'Bill Callahan', 'YTILAER')]

max_iter = 1 # one iteration is enough for most cases, raise it if errors occur
iter = 0
while iter < max_iter:
  new_tags = []
  for uid, singer, album in tags:
    # Remove dashes first
    singer = singer.replace("-", " ")
    album = album.replace("-", " ")
    try:
      # Try url first
      url = "https://rateyourmusic.com/release/album/%s/%s/" % (lower_alnum_dash(singer), lower_alnum_dash(album))
      tag = singer + " - " + album
      album_infos = network.get_album_infos(url=url)
    except:
      album_infos = []
    
    if not album_infos:
      try:
        # Try search then
        url_tag = urllib.parse.quote(tag)
        album_infos = network.get_album_infos(name=url_tag) # get album by search
      except IndexError as e:
        print("❌ ", tag)
      
    if not album_infos:
      new_tags.append((uid, singer, album))
    else:
      print("✅ ", tag)
      album_infos["Id"] = uid
      df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
    # except IndexError:
    # except (IndexError, AttributeError):

  tags = new_tags
  iter += 1

❌  Steve Reich - Pulse/Quartet
❌  Christina Vantzou - No. 4
❌  Bill Callahan - YTILAER


In [25]:
tags

[('stevereich2018pultet', 'Steve Reich', 'Pulse/Quartet'),
 ('christinavantzou2018no4no4', 'Christina Vantzou', 'No. 4'),
 ('billcallahan2022ytiaer', 'Bill Callahan', 'YTILAER')]

In [17]:
xdf = df.reset_index()
xdf = df[cols]
xdf.reset_index()
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages,Ranked
count,2272,2272,2272,2272,2270,2272,2272,1777,2272,2272,82,1813
unique,2269,2260,1763,333,2245,2177,1967,13,2255,2192,31,1813
top,arca2021kiciii,Into the Blue,Taylor Swift,2 March 2018,3.29\n \n/ 5.00.5\n\n ...,"Trap, Southern Hip Hop\nPop Rap",,English,[],"[#7a7a7a, #cccccc]","English, Spanish","#58 for 2018, #8,307 overall"
freq,3,2,6,22,2,8,163,1757,17,77,25,1


In [18]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/opt/homebrew/anaconda3/envs/edav/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
xdf = xdf[['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme','Ranked']]
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked
count,2272,2272,2272,2272,2270,2272,2272,1859,2272,2272,1813
unique,2269,2260,1763,333,2245,2177,1967,44,2255,2192,1813
top,arca2021kiciii,Into the Blue,Taylor Swift,2 March 2018,3.29\n \n/ 5.00.5\n\n ...,"Trap, Southern Hip Hop\nPop Rap",,English,[],"[#7a7a7a, #cccccc]","#58 for 2018, #8,307 overall"
freq,3,2,6,22,2,8,163,1757,17,77,1


In [20]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [21]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [22]:
xdf

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked,Num_of_RYM_Ratings
0,noname2018room25,Room 25,Noname,14 September 2018,3.59,"Jazz Rap, Conscious Hip Hop\nNeo-Soul, Jazz Po...","female vocals, introspective, conscious, po...",English,"[Self, Blaxploitation, Prayer Song, Window, Do...","[#cccccc, #afaeaa, #c9c9c8, #9c9c9a, #b7b7b7, ...","#58 for 2018, #8,307 overall",7439
0,rolotomassi2018timyit,Time Will Die and Love Will Bury It,Rolo Tomassi,2 March 2018,3.53,"Post-Hardcore, Mathcore\nPost-Rock, Blackgaze,...","female vocals, anxious, ethereal, progressi...",English,"[Towards Dawn, Aftermath, Rituals, The Hollow ...","[#b5b4b4, #9b9ba3, #6a6979, #9d9d9f, #7c7b89, ...",#167 for 2018,2307
0,architects2018holell,Holy Hell,Architects,9 November 2018,3.47,"Metalcore\nDjent, Post-Hardcore","male vocals, death, heavy, passionate, ene...",English,"[Death Is Not Defeat, Hereafter, Mortal After ...","[#7a7a7a, #cccccc]",#240 for 2018,1408
0,robyn2018honney,Honey,Robyn,26 October 2018,3.44,"Dance-Pop, Electropop, House\nDeep House, Ambi...","female vocals, atmospheric, sensual, rhythm...",English,"[Missing U, Human Being, Because It's in the M...","[#da978f, #9a4739, #bb3623, #bba597, #7a5d3d, ...",#283 for 2018,3759
0,yvestumor2018safove,Safe in the Hands of Love,Yves Tumor,5 September 2018,3.61,"Neo-Psychedelia, Post-Industrial, Art Pop\nHyp...","psychedelic, eclectic, surreal, atmospheric...",English,"[Faith in Nothing Except in Salvation, Economy...","[#8d989c, #214d58, #6ea3d5, #29767c, #407c96]","#45 for 2018, #6,920 overall",9801
...,...,...,...,...,...,...,...,...,...,...,...,...
0,µziq2018chaish,Challenge Me Foolish,µ-Ziq,13 April 2018,3.12,"IDM\nDrill and Bass, Atmospheric Drum and Bass...","spring, female vocals, melodic, atmospheric",NaN,"[Inclement, Undone, Challenge Me Foolish, Bass...","[#6f6d6e, #bf806f, #b9b4d4, #a4633b, #bfbfbd, ...","#1,091 for archival",186
0,coheedandcambria2018vaxres,Vaxis – Act I: The Unheavenly Creatures,Coheed and Cambria,5 October 2018,3.29,"Alternative Rock, Progressive Rock\nPost-Hardc...","science fiction, romantic, male vocals, mel...",English,"[Prologue, The Dark Sentencer, Unheavenly Crea...","[#9f9f9d, #74482d, #b3813a, #a36f47]",#653 for 2018,822
0,snohaalegra2019ughain,"- Ugh, those feels again",Snoh Aalegra,16 August 2019,3.36,"Contemporary R&B, Neo-Soul\nTrip Hop, Pop Soul","love, female vocals",English,"[Here Now (Intro), I Want You Around, Situatio...","[#7a7a7a, #cccccc]",#595 for 2019,496
0,sharonjonesthedapkings2020jussin,Just Dropped In (To See What Condition My Rend...,Sharon Jones and The Dap-Kings,23 October 2020,3.44,Southern Soul\nFunk,"female vocals, melodic, passionate, energetic",NaN,"[Signed Sealed Delivered I’m Yours, Little by ...","[#99a6b2, #614c37, #a6a092, #bcaf9a, #c68e81, ...","#51 for 2020, #5,811 overall",96


In [23]:
xdf.to_csv("rym_add.csv")